<a href="https://colab.research.google.com/github/lewirbi/GapfillingSentinel-2/blob/main/GapfillingSentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2021 Wirba P. Berinyuy { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Image Time Series Gapfilling on Sentinel-2 Using s2cloudless**

**Initialize the Earth Engine API**

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=K6M9-UOVuYz50ZEbMJOThd-glxE0KILwD4rSZPEhQNQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4lSL_5zkvbbrVRoXq4KhMu2XOh6ATy_B2fX6VfrXlCK6_-cVGLRsY

Successfully saved authorization token.


build an S2 SR collection and defines functions to add cloud and cloud shadow component layers to each image in the collection.

The values currently set for AOI, START_DATE, END_DATE, and CLOUD_FILTER are intended to build a collection for a single S2 overpass of a region 

In [45]:
AOI = ee.Geometry.Point(14.321, 10.605)
START_DATE = '2020-08-01'
END_DATE = '2020-08-02'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

Define a function to filter the SR and s2cloudless collections according to area of interest and date parameters, then join them on the system:index property. The result is a copy of the SR collection where each image has a new 's2cloudless' property whose value is the corresponding s2cloudless image.

In [46]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

separate the S2 SR and s2cloudless collections and see their properties

In [47]:
def get_s2_sr(aoi, start_date, end_date):
    # Import and filter S2 SR.
   s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
   return s2_sr_col

In [48]:
def get_cld_col(aoi, start_date, end_date):
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    return s2_cloudless_col

# print properties of collected **images**
properties of the S2 SR images

In [49]:
import pprint

# Load an image.
collection = get_s2_sr(AOI, START_DATE, END_DATE)

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
# Get the number of images.
count = collection.size()
print('Count: ', str(count.getInfo())+'\n')

# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', str(ee.Date(range.get('min')).getInfo()), str(ee.Date(range.get('max')).getInfo())+'\n')

# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
pp.pprint('Sun elevation statistics: '+str(sunStats.getInfo())+'\n')

# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', str(image.getInfo())+'\n')

# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
#pp.pprint('Recent images: '+str(recent.getInfo())+'\n')

# Get size of collection in bytes
print('Total size of Collection with 10 most recent images : '+str((recent.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))


Count:  1

Date range:  {'type': 'Date', 'value': 1596274673478} {'type': 'Date', 'value': 1596274673478}

("Sun elevation statistics: {'max': None, 'mean': 0, 'min': None, 'sample_sd': "
 "0, 'sample_var': 0, 'sum': 0, 'sum_sq': 0, 'total_count': 0, 'total_sd': 0, "
 "'total_var': 0, 'valid_count': 0, 'weight_sum': 0, 'weighted_sum': 0}\n")
Least cloudy image:  {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32633', 'crs_transform': [60, 0, 399960, 0, -60, 1200000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32633', 'crs_transform': [10, 0, 399960, 0, -10, 1200000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32633', 'crs_transform': [10, 0, 399960, 0, -10, 1200000]}, {'id': 'B4', 'data_ty

properties of the s2cloudless

In [50]:
import pprint

# Load an image.
collection = get_cld_col(AOI, START_DATE, END_DATE)

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
# Get the number of images.
count = collection.size()
print('Count: ', str(count.getInfo())+'\n')

# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', str(ee.Date(range.get('min')).getInfo()), str(ee.Date(range.get('max')).getInfo())+'\n')

# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
pp.pprint('Sun elevation statistics: '+str(sunStats.getInfo())+'\n')

# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', str(image.getInfo())+'\n')

# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
#pp.pprint('Recent images: '+str(recent.getInfo())+'\n')

# Get size of collection in bytes
print('Total size of Collection with 10 most recent images : '+str((recent.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))


Count:  1

Date range:  {'type': 'Date', 'value': 1596273631000} {'type': 'Date', 'value': 1596273631000}

("Sun elevation statistics: {'max': None, 'mean': 0, 'min': None, 'sample_sd': "
 "0, 'sample_var': 0, 'sum': 0, 'sum_sq': 0, 'total_count': 0, 'total_sd': 0, "
 "'total_var': 0, 'valid_count': 0, 'weight_sum': 0, 'weighted_sum': 0}\n")
Least cloudy image:  {'type': 'Image', 'bands': [{'id': 'probability', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32633', 'crs_transform': [10, 0, 399960, 0, -10, 1200000]}], 'version': 1596541420587491, 'id': 'COPERNICUS/S2_CLOUD_PROBABILITY/20200801T092031_20200801T093641_T33PVM', 'properties': {'system:time_start': 1596273631000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[15.089263855546832, 10.855512357693323], [15.089252437346614, 10.855513010815843], [14.0847795510916, 10.854165220760878], [14.084737459464769, 10.85412852852655], [14.084691025768963

Apply the get_s2_sr_cld_col function to build a collection according to the parameters defined above

In [51]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# **Define cloud mask component functions**

**Cloud components**
Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.

In [52]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

**Cloud shadow components**
Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. Note that the image input needs to be the result of the above add_cloud_bands function because it relies on knowing which pixels are considered cloudy ('clouds' band).

In [53]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.

In [54]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

**Define functions to display image and mask component layers.**
Folium will be used to display map layers. Import folium and define a method to display Earth Engine image tiles.

In [55]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

Display all of the cloud and cloud shadow components to an interactive Folium map. The input is an image collection where each image is the result of the add_cld_shdw_mask function defined previously.

In [56]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

# Display mask component layers
Map the add_cld_shdw_mask function over the collection to add mask component bands to each image, then display the results.

In [57]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

display_cloud_layers(s2_sr_cld_col_eval_disp)

# **Image Gapfilling to obtain cloudless composite image**

# Define collection filter and cloud mask parameters
We'll redefine the parameters to be a little more aggressive, i.e. decrease the cloud probability threshold, increase the cloud projection distance, and increase the buffer. These changes will increase cloud commission error (mask out some clear pixels), but since we will be compositing images from three months, there should be plenty of observations to complete the mosaic.

In [59]:
AOI = ee.Geometry.Point(14.321, 10.605)
START_DATE = '2018-05-01'
END_DATE = '2018-12-02'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

check the information on the new collection

In [70]:
import pprint

# Load an image.
collection2 = get_cld_col(AOI, START_DATE, END_DATE)

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
# Get the number of images.
count = collection2.size()
print('Count: ', str(count.getInfo())+'\n')

# Get the date range of images in the collection.
range = collection2.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', str(ee.Date(range.get('min')).getInfo()), str(ee.Date(range.get('max')).getInfo())+'\n')

# Get statistics for a property of the images in the collection.
sunStats = collection2.aggregate_stats('SUN_ELEVATION')
pp.pprint('Sun elevation statistics: '+str(sunStats.getInfo())+'\n')

# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection2.sort('CLOUD_COVER').first())
print('Least cloudy image: ', str(image.getInfo())+'\n')

# Limit the collection to the 1000 most recent images.
recent = collection2.sort('system:time_start', False).limit(1000)
#pp.pprint('Recent images: '+str(recent.getInfo())+'\n')

# Get size of collection in bytes
print('Total size of Collection with 10 most recent images : '+str((recent.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))


Count:  46

Date range:  {'type': 'Date', 'value': 1525425631000} {'type': 'Date', 'value': 1543569811000}

("Sun elevation statistics: {'max': None, 'mean': 0, 'min': None, 'sample_sd': "
 "0, 'sample_var': 0, 'sum': 0, 'sum_sq': 0, 'total_count': 0, 'total_sd': 0, "
 "'total_var': 0, 'valid_count': 0, 'weight_sum': 0, 'weighted_sum': 0}\n")
Least cloudy image:  {'type': 'Image', 'bands': [{'id': 'probability', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32633', 'crs_transform': [10, 0, 399960, 0, -10, 1200000]}], 'version': 1584941348656403, 'id': 'COPERNICUS/S2_CLOUD_PROBABILITY/20180504T092031_20180504T092516_T33PVM', 'properties': {'system:time_start': 1525425631000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[15.089263855546832, 10.855512357693323], [15.089252437346614, 10.855513010815843], [14.0847795510916, 10.854165220760878], [14.084737459464769, 10.85412852852655], [14.08469102576896

# Build a Sentinel-2 collection
Reassemble the S2-cloudless collection since the collection filter parameters have changed.

In [62]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

**Define a function to apply the cloud mask to each image in the collection.**

In [63]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

Add cloud and cloud shadow component bands to each image and then apply the mask to each image. Reduce the collection by median (in your application, you might consider using medoid reduction to build a composite from actual data values, instead of per-band statistics).

In [68]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

Display properties of the composite image

**Display the cloud-free composite**
Display the results. Be patient while the map renders, it may take a minute; ee.Image.reproject is forcing computations to happen at 100 and 20 m scales (i.e. it is not relying on appropriate pyramid level scales for analysis). The issues with ee.Image.reproject being resource-intensive in this case are mostly confined to interactive map viewing. Batch image exports and table reduction exports where the scale parameter is set to typical Sentinel-2 scales (10-60 m) are less affected.

In [69]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

# END